In [12]:
import time
import httpx
import pandas as pd

url = "https://www.reddit.com/r/datacenter/new.json"

dataset = []

params = {
    'limit': 100,
    't': 'month',
    'after': None
}

response = httpx.get(url, params=params)
print(f'fetching "{response.url}"...')

json_data = response.json()

children = json_data.get("data", {}).get("children", [])

dataset.extend([rec['data'] for rec in children])

df = pd.DataFrame(dataset)

df = df[["selftext", "title"]]
df = df.dropna(subset = ["selftext"])

df.head(15)

fetching "https://www.reddit.com/r/datacenter/new.json?limit=100&t=month&after="...


,selftext,title
0,,Microsoft CET Interview for Toronto Data Cente...
1,,"If the stock market crashes, will it affect th..."
2,"Hi folks,\n\nI recently joined a fast growing ...",How good is Cadence's Digital Twin?
3,\nI’ve had recruiters reach out to me over the...,Recruiters Reaching Out to You for a Contract ...
4,What are some good data center technician cert...,What are good data center technician certs to ...
5,"Hi everyone,\n\n \nI am a sophomore at UW-Mad...",Jobs in Data centers/Infrastructure
6,There is a position available near me (Frederi...,Amazon health insurance
7,"Hey everyone,\nI’m 25 and trying to figure out...",Breaking into Data Center Careers at 25 — What...
8,I’m 21 and want to get into Data Center Techni...,Rookie need help
9,I work for one of the hyper scalers on the des...,Is the pace of delivery getting this bad at ot...


In [10]:
from transformers import pipeline
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)

/Users/mnatali/Projects/sentiment_analysis/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


In [14]:
texts = df["selftext"].astype(str).tolist()
results = classifier(texts, truncation=True, padding=True, max_length=512, batch_size=32)
df["roberta"] = results

def list_to_dict(list):
    return list[0]

df["roberta1"] = df["roberta"].apply(list_to_dict)

KeyboardInterrupt: 

In [ ]:
df["name"] = df["roberta1"].apply(lambda x: x["label"])

def roberta_label(name):
    if name == 'LABEL_2':
        return "positive"
    elif name == 'LABEL_1':
        return "neutral"
    else:
        return "negative"
df["label"] = df["name"].apply(roberta_label)

df = df[["selftext", "title", "label"]]